# Data Preprocessing

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")
df = pd.read_csv('default_payment_dataset.csv', sep=';')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,25150,360000,2,2,2,25,-1,-1,-1,-1,...,3435,8870,2020,12590,3479,3446,8870,2020,27043,0
1,13470,240000,2,1,2,46,2,2,-2,-1,...,2240,1681,2267,0,0,2240,0,2267,3074,0
2,3092,320000,2,2,1,41,0,0,0,0,...,35409,35036,35939,5000,2000,2000,3000,3000,1500,0
3,13973,50000,2,2,2,24,-1,-1,-2,-2,...,0,0,0,0,0,0,0,0,0,1
4,10567,80000,1,3,1,52,-1,0,0,0,...,20561,21261,21457,1500,1500,1500,1028,876,1449,0


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
X = df.drop(columns=['ID','SEX','EDUCATION','MARRIAGE','AGE','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                     'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6','default_payment_next_month'])
y = df['default_payment_next_month']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [1]:
# print(range(len(X_train.columns)))
# X_train.iloc[:,0]

# Data Cleansing

## Handle Missing Value

Tidak di temukan missing value pada dataset yang digunakan.

In [21]:
df['EDUCATION'].value_counts()

2    9789
1    7415
3    3467
4     329
Name: EDUCATION, dtype: int64

## Handle Duplicate Data

In [22]:
df['ID'].duplicated().sum()

0

Tidak ditemukan duplicated di kolom ID pada dataset yang digunakan

## Handle Outliers

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          21000 non-null  int64
 1   LIMIT_BAL                   21000 non-null  int64
 2   SEX                         21000 non-null  int64
 3   EDUCATION                   21000 non-null  int64
 4   MARRIAGE                    21000 non-null  int64
 5   AGE                         21000 non-null  int64
 6   PAY_0                       21000 non-null  int64
 7   PAY_2                       21000 non-null  int64
 8   PAY_3                       21000 non-null  int64
 9   PAY_4                       21000 non-null  int64
 10  PAY_5                       21000 non-null  int64
 11  PAY_6                       21000 non-null  int64
 12  BILL_AMT1                   21000 non-null  int64
 13  BILL_AMT2                   21000 non-null  int64
 14  BILL_A

In [7]:
from scipy import stats

X_y_train = X_train.join(y_train)

for i in range(0, len(X_y_train.columns)):
    z_scores = np.abs(stats.zscore(X_y_train.iloc[:,i]))
    filtered_entries = (z_scores < 3)
    X_y_train= X_y_train[filtered_entries]

X_y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16377 entries, 15345 to 15795
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   16377 non-null  int64
 1   PAY_0                       16377 non-null  int64
 2   PAY_2                       16377 non-null  int64
 3   PAY_3                       16377 non-null  int64
 4   PAY_4                       16377 non-null  int64
 5   PAY_5                       16377 non-null  int64
 6   PAY_6                       16377 non-null  int64
 7   default_payment_next_month  16377 non-null  int64
dtypes: int64(8)
memory usage: 1.1 MB


Hasil sebaran data numerik menggunakan boxplot setelah dilakukan filterisasi outliers.<br>
Setelah semua outliers di filter dengan menggunakan z-score, tersisa **16.830** data dari **21.000**

## Data Transformation

Tidak perlu dilakukan data transformation karena:<br>
1. Skala pada fitur-fitur yang akan digunakan sudah seragam (-2 s/d 8) (Normalization dan Standardization)
2. Rentang nilai pada fitur yang akan digunakan juga tidak besar (Log Transformation)
3. Pada fitur yang akan digunakan, terdapat nilai negative dan nol, sehingga kurang cocok jika dilakukan proses Log Transformation

## Data Encoding

Semua data pada dataset sudah di encode ke dalam bentuk numeric

## Handle Class Imbalance

In [8]:
from imblearn.over_sampling import SMOTE

X_train = X_y_train.drop(columns=['default_payment_next_month'])
y_train = X_y_train['default_payment_next_month']

sm = SMOTE(random_state=42)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

Terjadi *class imbalance* pada fitur target, sehingga dilakukan **oversampling** terhadap fitur tersebut dengan menggunakan metode **SMOTE**

# Feature Engineering

## Feature Selection

In [9]:
# Metode SelectKBest untuk menentukan fitur-fitur penting
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# Buat SelectKBest object
sel = SelectKBest(f_classif, k=4)

# Fit dan transform data
X_new = sel.fit_transform(X_sm, y_sm)

# Mendapatkan nama kolom fitur terbaik
selected_features = X_sm.columns[sel.get_support()]
X_sm[selected_features].columns

Index(['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4'], dtype='object')

Proses **Feature Selection** dilakukan dengan menggunakan metode **SelectKBest**, dimana dipilih 4 (k=4) fitur terbaik berdasarkan dari perhitungan statistik metode SelectKBest. Adapun fitur-fitur yang didapatkan:<br>
1. PAY_0
2. PAY_2
3. PAY_3
4. PAY_4

## Feature Extraction

In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=1)
X_lda = lda.fit_transform(X_sm[selected_features], y_sm)

X_new = pd.DataFrame(data=X_sm[selected_features])
X_new['ld1'] = X_lda

X_new.head()

,PAY_0,PAY_2,PAY_3,PAY_4,ld1
0,1,-2,-2,-2,-0.114854
1,1,-2,-2,-2,-0.114854
2,0,0,0,0,0.115848
3,-1,-1,-1,-1,1.141293
4,0,0,0,0,0.115848


Pada feature extraction digunakan metode **Linear Discriminant Analysis (LDA)**, dimana metode ini akan menghasilkan sebuah prediksi dengan memperhitungkan input dari masing-masing class yang digunakan.<br>

Karena algoritma yang akan digunakan adalah Decision Tree, LDA digunakan untuk memilih fitur-fitur yang paling baik membedakan antara kelas label pada dataset, sehingga dapat meningkatkan akurasi dan performa decision tree

## Ide Feature Tambahan

- Banyaknya **penghasilan nasabah**. Penghasilan nasabah kemungkinan memengaruhi limit balance yang diberikan sehingga bisa jadi memengaruhi pula status nasabah tersebut akan melakukan default atau tidak. LImit balance yang tinggi bisa jadi berasal dari penghasilan yang tinggi sehingga outlier yang terjadi di fitur limit balance belun tentu harus dihilangkan
- **Jumlah tanggungan/anggota keluarga**. Semakin banyak orang yang menjadi tanggungan nasabah maka semakin banyak juga pengeluaran untuk kebutuhan hidup sehari-sehari termasuk penggunaan kartu kredit yang menjadi lebih besar. Banyaknya pengeluaran dan tingginya tagihan kartu kredit yang diterima dapat membuat nasabah tersebut melakukan default karen atidak sanggup membayar tagihan
- **Status pengguna baru**. Apakah nasabah tersebut termasuk pengguna baru atau bukan. Jika seseorang baru memiliki kartu kredit dan belum memiliki riwayat pembayaran yang baik, maka bank mungkin akan merasa kurang yakin untuk memberikan kredit yang besar
- **Jenis Pekerjaan**. Jenis pekerjaan nasabah juga dapat memengaruhi kemungkinan nasabah untuk mengalami default. Jika nasabah bekerja di bidang yang tidak stabil atau memiliki risiko kehilangan pekerjaan, maka kemungkinan nasabah sulit membayar tagihan kartu kredit dan akhirnya mengalami default
- **Jumlah tagihan terhutang** yang bisa didapat dari selisih antara bill amount dan pay amount. Ketika pay amount tidak sesuai dengan bill amount, maka sisa bill ampunt yang masih belum terbayarkan akan terakumulasi di bulan selanjutnya. Sehingga ketika nasabah melakukan pembayran kurang dari tagihan rentan mengalami default karena jumlah tagihannya akan terus bertambah